Lessons learnt
1. View the csv files in Catalog-> Browse DBFS
2. View Delata table in Catalog -> databricks_ml_ws -> default-> tables
3. Remove .(dot) in file headers

In [0]:
from pyspark.sql.types import DoubleType, StringType, StructType, StructField

schema = StructType([
    StructField("age", DoubleType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", DoubleType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", DoubleType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", DoubleType(), True),
    StructField("capital_loss", DoubleType(), True),
    StructField("hours_per_week", DoubleType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True)
])
census_df = spark.read.csv("/FileStore/tables/Census_income_t.csv",schema=schema)  

In [0]:
census_df.display()

In [0]:
train_df, test_df = census_df.randomSplit([0.8, 0.2], seed=12345)

In [0]:
from databricks import automl

In [0]:
summary = automl.classify(train_df, target_col="income", timeout_minutes=5)

In [0]:
#Method 1 - Predict using Pandas Df

model_uri = summary.best_trial.model_path

import mlflow

#Preparing test df
test_p_df = test_df.toPandas()

y_test = test_p_df["income"]
x_test = test_p_df.drop(["income"], axis=1)

#Run inference using best model
model= mlflow.pyfunc.load_model(model_uri)
predict = model.predict(x_test)
test_p_df["income_Predicted"] = predict
display(test_p_df)

In [0]:
#Method 2: Predict using Spart user defined function

import mlflow

model_uri = f"runs:/{summary.best_trial.mlflow_run_id}/model"

predict = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri, result_type="String")
pred_df = test_df.withColumn("Income_Predicted", predict(*test_df.drop("income").columns))
display(pred_df)

In [0]:
#Evaluate the model

import sklearn.metrics

model = mlflow.sklearn.load_model(model_uri)

